## PROBLEM 1: KMeans Theory

Kmeans Objective: min(sum(sum(pi_ik * ||X_i - M_k||^2))), where pi_ik = 1 if X_i is assigned to cluster k, 0 otherwise.
(A) Proof E step(updating membership minimize the objective)
        Given the current centroids, we can find the closest M_k to X_i and assign the X_i to the cluster making the objective minimum.
(B) Proof M step(updating centroids minimize the objective)
       Given the current membership, we can calculate the derivative of the objective function with respect to the centroids and set it to zero to find the optimal centroids. 
       After differentiation, the objective function can be written as sum(pi_ik * (2M_k - 2X_i)) = 0, 
                                                                                                        M_k * sum(pi_ik) = sum(pi_ik*X_i)
                                                                                                        M_k = sum(pi_ik*X_i)/sum(pi_ik).
       Therefore, the solution of M_k to minimize the objective function is the average of the data points in the cluster k.   
(C) By the proof of (A) and (B), the iteration of the KMeans algorithm make the objective function decrease or unchanged monotonically. And it will converge to a point when no further decrease is possible. The E and M steps might converge at a local minimum, which also fulfills the goal of no further decrease.

## PROBLEM 2 : KMeans on data

In [42]:
from sklearn.metrics import pairwise_distances
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.datasets import fashion_mnist
from keras.datasets import mnist
import numpy as np


In [35]:
def initialize_centroids(X, K):
    '''
    randomly select k data points as the initial centroids
    :param X: data
    :param K: number of clusters
    :return: list of K centroids
    '''
    np.random.seed(17)
    # randomly select K indices, use X[indices] as the initial centroids
    indices = np.random.choice(X.shape[0], K, replace=False) 
    return X[indices]

def hard_kmeans(X, K, max_iter = 100, tol = 1e-4):
    '''
    :param X: data
    :param K: number of clusters
    :param max_iter: maximum number of iterations
    :param tol: tolerance for stopping criterion
    :return:  assignments, centroids, objective function value
    '''
    centroids = initialize_centroids(X, K)
    for i in range(max_iter):
        # E step
        dist = pairwise_distances(X, centroids)
        assignments = np.argmin(dist, axis=1)
        # M step
        new_centroids = np.array([X[assignments == k].mean(axis=0) for k in range(K)])
        if np.linalg.norm(new_centroids - centroids) < tol:
            break
        centroids = new_centroids

    obj = np.sum((X - centroids[assignments])**2)
    return assignments, centroids, obj


def normalize_images(X):
    normalized_images = []
    X = X.reshape(X.shape[0], X.shape[1]*X.shape[2])
    for image in X:
        normalized_images.append((image - np.min(image))/np.max(image))
    return np.array(normalized_images)

def soft_kmeans(X, K, beta, max_iter = 100, tol = 1e-4):
    '''
    :param X: data
    :param K: number of clusters
    :param beta: parameter for soft kmeans
    :param max_iter: maximum number of iterations
    :param tol: tolerance for stopping criterion
    :return:  assignments, centroids,objective function value
    '''
    if np.isnan(X).any():
        means = np.nanmean(X, axis=0)
        X = np.where(np.isnan(X), means, X) # replace NaN with the mean of the column
    centroids = initialize_centroids(X, K)
    for i in range(max_iter):
        # E step
        dist = pairwise_distances(X, centroids)
        assignments = np.exp(-beta * dist)
        assignments /= (assignments.sum(axis=1)[:, np.newaxis]+ 1e-10)
        # M step
        new_centroids = np.dot(assignments.T, X) / (assignments.sum(axis=0)[:, np.newaxis]  + 1e-10)
        if np.linalg.norm(new_centroids - centroids) < tol:
            break
        centroids = new_centroids
    obj = np.sum(np.sum(assignments * dist))
    return assignments, centroids, obj

def purity(assignments, true_labels, K):
    '''
    external measure for clustering by purity:
    formula: sum()/n
    :param assignments: cluster assignments
    :param true_labels: true labels
    :return: purity
    '''
    n = len(assignments)
    purity = 0
    for k in range(K):
        indices = np.where(assignments == k)[0]
        if len(indices) == 0:
            continue
        counts = np.bincount(true_labels[indices]) # count the occurrence of each label
        purity += np.max(counts) # count of the most frequent label
    return purity / n

def gini_index(assignments, true_labels, K):
    '''
    external measure for clustering by gini index:
    formula: 1 - sum(p_k * (1 - sum(p_i^2))), where is probability distribution of the true labels within a given cluster k
    :param assignments: cluster assignments
    :param true_labels: true labels
    :return: gini index
    '''
    n = len(assignments)
    gini = 0
    for k in range(K):
        indices = np.where(assignments == k)[0]
        if len(indices) == 0:
            continue
        counts = np.bincount(true_labels[indices]) # count the occurrence of each label
        p_k = counts / counts.sum() # proportion of k-th class
        gini_k = 1 - np.sum(p_k ** 2)
        gini += (len(indices) / n) * gini_k # weight by cluster size
    return gini

In [11]:
#A) MNIST Dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_test_norm = normalize_images(X_test)

In [33]:
#evaluate the KMeans objective for a higher K (for example double) or smaller K(for example half).
print("MNIST Dataset internal measures")
assignments_5, _, obj_5 = hard_kmeans(X_test_norm, 5)
print("K=5, Objective: ", obj_5)
assignments_10, _, obj_10 = hard_kmeans(X_test_norm, 10)
print("K=10, Objective: ", obj_10)
assignments_20, _, obj_20 = hard_kmeans(X_test_norm, 20)
print("K=20, Objective: ", obj_20)

MNIST Dataset internal measures
K=5, Objective:  432282.69567812496
K=10, Objective:  392102.3941812986
K=20, Objective:  349432.15135823673


In [37]:
print("MNIST Dataset external measures")
print(" K=5, Purity: ", purity(assignments_5, y_test, 5),"  ", "Gini Index: ", gini_index(assignments_5, y_test, 5))
print(" K=10, Purity: ", purity(assignments_10, y_test, 10),"  ", "Gini Index: ", gini_index(assignments_10, y_test, 10))
print(" K=20, Purity: ", purity(assignments_20, y_test, 20),"  ", "Gini Index: ", gini_index(assignments_20, y_test, 20))

MNIST Dataset external measures
 K=5, Purity:  0.4525    Gini Index:  0.6585887450133034
 K=10, Purity:  0.5272    Gini Index:  0.5936870630149979
 K=20, Purity:  0.7204    Gini Index:  0.38698754685545056


In [40]:
print("MNIST Dataset soft KMeans")
soft_assignments1, _, obj1 = soft_kmeans(X_test_norm, 10, 0.1)
print("Beta=0.1, Objective: ", obj1, "  ", "Purity: ", purity(np.argmax(soft_assignments1, axis=1), y_test, 10), "  ", "Gini Index: ", gini_index(np.argmax(soft_assignments1, axis=1), y_test, 10))
soft_assignments2, _, obj2 = soft_kmeans(X_test_norm, 10, 1)
print("Beta=1, Objective: ", obj2, "  ", "Purity: ", purity(np.argmax(soft_assignments2, axis=1), y_test, 10), "  ", "Gini Index: ", gini_index(np.argmax(soft_assignments2, axis=1), y_test, 10))
soft_assignments3, _, obj3 = soft_kmeans(X_test_norm, 10, 10)
print("Beta=10, Objective: ", obj3, "  ", "Purity: ", purity(np.argmax(soft_assignments3, axis=1), y_test, 10), "  ", "Gini Index: ", gini_index(np.argmax(soft_assignments3, axis=1), y_test, 10))

MNIST Dataset soft KMeans
Beta=0.1, Objective:  72088.81146261489    Purity:  0.2733    Gini Index:  0.8298867835172213
Beta=1, Objective:  72088.81145907445    Purity:  0.2105    Gini Index:  0.869986882809169
Beta=10, Objective:  61581.90211387066    Purity:  0.5306    Gini Index:  0.5894889401892156


In [43]:
#B) FASHION Dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_test_norm = normalize_images(x_test)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [48]:
np.bincount(y_test)

array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000],
      dtype=int64)

In [47]:
print("fashion_mnist Dataset measures")
assignments_5, _, obj_5 = hard_kmeans(X_test_norm, 5)
print(" K=5, Purity: ", purity(assignments_5, y_test, 5),"  ", "Gini Index: ", gini_index(assignments_5, y_test, 5), "  ", "Objective: ", obj_5)
assignments_10, _, obj_10 = hard_kmeans(X_test_norm, 10)
print(" K=10, Purity: ", purity(assignments_10, y_test, 10),"  ", "Gini Index: ", gini_index(assignments_10, y_test, 10), "  ", "Objective: ", obj_10)
assignments_20, _, obj_20 = hard_kmeans(X_test_norm, 20)
print(" K=20, Purity: ", purity(assignments_20, y_test, 20),"  ", "Gini Index: ", gini_index(assignments_20, y_test, 20), "  ", "Objective: ", obj_20)

fashion_mnist Dataset measures
 K=5, Purity:  0.1082    Gini Index:  0.8997557926683448    Objective:  432282.69567812496
 K=10, Purity:  0.1134    Gini Index:  0.8990142616258867    Objective:  392102.3941812986
 K=20, Purity:  0.1199    Gini Index:  0.89831345889829    Objective:  349479.4016943277


In [66]:
#C) 20NG Dataset

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

test_data = fetch_20newsgroups(subset='test', shuffle=True, random_state=17)

# Apply TF normalization
vectorizer = TfidfVectorizer()
test_documents_tfidf = vectorizer.fit_transform(test_data.data).toarray()
y_test = test_data.target

In [67]:
print("20NG Dataset measures")
assignments_10, _, obj_10 = hard_kmeans(test_documents_tfidf, 10)
print(" K=10, Purity: ", purity(assignments_10, y_test, 10),"  ", "Gini Index: ", gini_index(assignments_10, y_test, 10), "  ", "Objective: ", obj_10)
assignments_20, _, obj_20 = hard_kmeans(test_documents_tfidf, 20)
print(" K=20, Purity: ", purity(assignments_20, y_test, 20),"  ", "Gini Index: ", gini_index(assignments_20, y_test, 20), "  ", "Objective: ", obj_20)
assignments_30, _, obj_30 = hard_kmeans(test_documents_tfidf, 30)
print(" K=30, Purity: ", purity(assignments_30, y_test, 30),"  ", "Gini Index: ", gini_index(assignments_30, y_test, 30), "  ", "Objective: ", obj_30)


20NG Dataset measures
 K=10, Purity:  0.21216144450345195    Gini Index:  0.8713301810616431    Objective:  7005.268149600954
 K=20, Purity:  0.20379713223579396    Gini Index:  0.8611020325434653    Objective:  6961.470507827989
 K=30, Purity:  0.22039298990971853    Gini Index:  0.8398194867573759    Objective:  6895.8515665928335


## PROBLEM 3 : Gaussian Mixture on toy data

In [68]:
from scipy.stats import multivariate_normal
import numpy.linalg as LA

In [94]:
def initialize_parameters(X, K):
    '''
    randomly select k data points as the initial centroids
    :param X: data
    :param K: number of clusters
    '''
    np.random.seed(17)
    N, d = X.shape
    weights = np.ones(K) / K
    
    # choose K random data points as the initial means
    indices = np.random.choice(N, K, replace=False)
    means = X[indices]
    
    # initialize covariances.
    data_cov = np.cov(X, rowvar=False)
    covariances = np.array([data_cov.copy() for _ in range(K)]) # 1 covariance matrix per cluster
    return weights, means, covariances

def logsumexp(a, axis, keepdims=True):
    '''
    computing the log-sum-exp of the array 'a' in a numerically stable way
    '''
    a_max = np.max(a,axis = axis, keepdims=True)
    s = np.log(np.sum(np.exp(a - a_max), axis=axis, keepdims=True))
    if not keepdims:
        s = np.squeeze(s, axis=axis)
        a_max = np.squeeze(a_max, axis=axis)
    return a_max + s

def gmm(X, K, max_iter=100, tol = 1e-6, eps=1e-6):
    N,d = X.shape
    weights, means, covariances = initialize_parameters(X, K)
    log_likelihoods = []
    for i in range(max_iter):
        #E-step
        log_resp = np.zeros((N,K))
        for k in range(K):
            #add small constant to the diagonal
            cov_k = covariances[k] + np.eye(d) * eps
            #compute log probability for each point under the k-th Gaussian
            log_pdf = multivariate_normal.logpdf(X, mean=means[k], cov=cov_k)
            log_resp[:,k] = np.log(weights[k]) + log_pdf
            
        #use log-sum-exp to get the log likelihood and normalize resp
        log_sum = logsumexp(log_resp,axis=1,keepdims=True)
        log_likelihood = np.sum(log_sum)
        log_likelihoods.append(log_likelihood)
        
        #convert log resp to actual resp
        resp = np.exp(log_resp - log_sum) #shape (N,K)
        
        #M-step
        Nk = resp.sum(axis=0) # effective number of points per component, shape (K,)
        weights = Nk/N 
        new_means = np.zeros((K, d))
        new_covariances = np.zeros((K, d, d))
        for k in range(K):
            new_means[k] = (resp[:,k][:,np.newaxis] * X).sum(axis=0) / Nk[k]
            diff = X - new_means[k]
            new_covariances[k] = (resp[:, k][:, np.newaxis] * diff).T.dot(diff) / Nk[k]
            new_covariances[k] += np.eye(d) * eps
        means, covariances = new_means, new_covariances
        
        #convergence check
        if i > 0 and np.abs(log_likelihood - log_likelihoods[-2]) < tol:
            print(f"Converged after {i} iterations")
            break
        
    return weights, means, covariances, log_likelihoods

In [97]:
g2 = np.loadtxt("dataset/2gaussian.txt")
weights, means, covariances, log_likelihoods = gmm(g2, 2)
print("Weights: ", weights)
print("Means: ", means)
print("Covariances: ", covariances)
#print("Log likelihoods: ", log_likelihoods)

Converged after 26 iterations
Weights:  [0.66520194 0.33479806]
Means:  [[7.0131553  3.98313827]
 [2.99414548 3.05209487]]
Covariances:  [[[0.97474789 0.49746329]
  [0.49746329 1.00113766]]

 [[1.01025806 0.02718868]
  [0.02718868 2.93781334]]]


In [98]:
g3 = np.loadtxt("dataset/3gaussian.txt")
weights, means, covariances, log_likelihoods = gmm(g3, 3)
print("Weights: ", weights)
print("Means: ", means)
print("Covariances: ", covariances)
#print("Log likelihoods: ", log_likelihoods)

Converged after 73 iterations
Weights:  [0.29843544 0.20560498 0.49595958]
Means:  [[7.0215704  4.01546579]
 [3.03973175 3.04859572]
 [5.01174254 7.0014849 ]]
Covariances:  [[[0.99039914 0.50095338]
  [0.50095338 0.99564928]]

 [[1.02853436 0.02689709]
  [0.02689709 3.38491062]]

 [[0.97969752 0.18514711]
  [0.18514711 0.97452846]]]


## PROBLEM 4 EM for coin flips

In [120]:
from scipy.special import comb, logsumexp

def binomial_mixture(heads, D, K, max_iter=100, tol=1e-6):
    N = len(heads)
    np.random.seed(17)
    #ensures sum(pi) = 1
    pi = np.random.dirichlet(alpha=np.ones(K)) 
    #randomly initialize p from 0.2 to 0.8 to avoid extreme values
    p = np.random.uniform(0.2,0.8,size=K)
    log_likelihoods = []
    
    for i in range(max_iter):
        #E-step
        logC = np.log(comb(D, heads))
        log_prob = np.zeros((N,K))
        for k in range(K):
            log_prob[:, k] = logC + heads * np.log(p[k]) + (D-heads) * np.log(1-p[k])
        log_prob += np.log(pi)
        
        log_sum = logsumexp(log_prob,axis=1)
        log_likelihood = np.sum(log_sum)
        log_likelihoods.append(log_likelihood)
        
        #compute reponsibilities: r_ik = exp(log_prob - log_sum_i)
        resp = np.exp(log_prob - log_sum[:,None])
        
        #M-step
        #effective number of sessions assigned to each coin, shape (K,)
        Nk = resp.sum(axis=0)
        pi = Nk/N
        
        #p_k = (weighted sum of heads) / (D * effective sessions)
        for k in range(K):
            p[k] = np.sum(resp[:,k] * heads) / (D * Nk[k])
        
        #convergence check
        if i > 0 and np.abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol:
            print(f"Converged after {i} iterations")
            break
            
    return pi, p, log_likelihoods
        
    
    

In [121]:
data = np.loadtxt("dataset/coin_flips_outcome.txt",dtype=int)
heads = np.sum(data,axis=1)
D = data.shape[1]
pi, p, log_likelihoods = binomial_mixture(heads, D, 3)
print("Estimated coin selection probabilities (pi):", pi)
print("Estimated coin head biases (p):", p)

Converged after 22 iterations
Estimated coin selection probabilities (pi): [0.30679528 0.17855886 0.51464586]
Estimated coin head biases (p): [0.23690868 0.93172817 0.61002961]


In [1]:
from sklearn.datasets import make_moons

x, y = make_moons(n_samples=1000, random_state=17)
x

array([[ 1.22777875e+00, -4.73712915e-01],
       [-1.00000000e+00,  1.22464680e-16],
       [ 1.90711910e+00,  7.91259922e-02],
       ...,
       [ 6.21378005e-01,  7.83510928e-01],
       [ 5.34885651e-02,  9.98568462e-01],
       [ 2.85145667e-01, -1.99273396e-01]])